# Introduction

**This demo shows to run a distributed simulation of protein folding using GROMACS within a bittensor subnet**

In this subnet:
- Validators select a protein (pbd_id), download the structure and preprare input files
- Miners run the simulation and send back their results
- Scoring is based on free energy of the folded structure

In [ ]:
import bittensor as bt

from folding.protocol import Synapse
from folding.miners.forward import forward
from folding.validators.protein import Protein
from typing import Tuple


bt.trace()


### Protein class is contains the protein sequence and the current state of the protein folding simulation.

In [ ]:
protein = Protein(max_steps=500)
protein

### validator is currently responsible for preparing the protein for the simulation.

In [ ]:
protein.md_inputs

# Simulation using only Synapse

In [ ]:
synapse = Synapse(pdb_id=protein.pdb_id, md_inputs=protein.md_inputs)#, mdrun_args='-maxh 0.01')
synapse

### Simulate the miner receiving the synapse and performing the md simulation.

In [ ]:
forward(synapse)

### Simulation results are attached to synapse

In [ ]:
synapse.md_output.keys()

### Perform reward calculation for miner

In [ ]:
reward = protein.reward(synapse.md_output)

reward

# Simulation using Dendrite

In [ ]:

def blacklist( synapse: Synapse ) -> Tuple[bool, str]:
    """
    Determines if the provided synapse should be blacklisted.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        Tuple[bool, str]: A tuple containing a boolean that indicates whether the synapse is blacklisted,
                          and a string providing the reason.
    """
    return False, ""

def priority( synapse: Synapse ) -> float:
    """
    Determines the priority of the provided synapse.

    Args:
        synapse (Prompting): The input synapse to be evaluated.

    Returns:
        float: The priority value of the synapse, with higher values indicating higher priority.
    """
    return 0.0

In [ ]:
# Create an Axon instance on port 8099.
axon = bt.axon(port=8098)

# Attach the forward, blacklist, and priority functions to the Axon.
# forward_fn: The function to handle forwarding logic.
# blacklist_fn: The function to determine if a request should be blacklisted.
# priority_fn: The function to determine the priority of the request.
axon.attach(
    forward_fn=forward,
    blacklist_fn=blacklist,
    priority_fn=priority
)

# Start the Axon to begin listening for requests.
axon.start()

# Create a Dendrite instance to handle client-side communication.
dendrite = bt.dendrite()

# Send a request to the Axon using the Dendrite, passing in a StreamPrompting instance with roles and messages.
# The response is awaited, as the Dendrite communicates asynchronously with the Axon.
resp = await dendrite(
    [axon],
    Synapse(pdb_id=protein.pdb_id, md_inputs=protein.md_inputs),
    deserialize=True
)

# The response object contains the result of the prompting operation.
resp

# Outlook
We have demonstrated that a molecular dynamics simulation can be carried out in the context of a subnet.

## Remaining Steps
- Use Gromacs python API
- Run on staging
- Run on testnet
- Run on mainnet


## Opportunities for Improvements
- Improved customization of input files (e.g. force field, box, mdp templates)
- Performance optimization (file usage, simulation length, parallelization)
- Allow for different miners (e.g. AI models versus GPU models versus CPU models)
- Perturbation of the structure (e.g. mutation) to prevent lookup attacks
- More complex scoring function (e.g. based on RMSD)
- More complex simulation (e.g. folding of a protein with multiple chains)